# Deploy and Run the Hamiltonian Simulation Function

This interactive guide shows how to upload the hamiltonian simulation function to Qiskit Serverless and run an example workload.

### Requirements

This guide was developed with the following package versions:

```
qiskit-ibm-catalog == 0.5.0
qiskit-ibm-runtime == 0.38.0
```

## 1. Autentication

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key (token), which you can find on the [IBM Quantum Platform](https://quantum.cloud.ibm.com) dashboard. This will allow you to locally instantiate the serverless client to upload or run the selected function:

```python
from qiskit_ibm_catalog import QiskitServerless
serverless = QiskitServerless(token="MY_TOKEN")
```

You can optionally use `save_account()` to save your credentials in your local environment (see the [Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-save) guide). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account):

```python
QiskitServerless.save_account(token="MY_TOKEN")
```

If the account is saved, there is no need to provide the token to authenticate:

In [1]:
from qiskit_ibm_catalog import QiskitServerless
# Authenticate to the remote cluster
serverless = QiskitServerless(channel="ibm_quantum")

## 2. Upload the Function

To upload a Qiskit Function, you must first instantiate a `QiskitFunction` object that defines the function source code. The title will allow you to identify the function once it's in the remote cluster. The main entry point is the file that contains `if __name__ == "__main__"`. If your workflow requires additional source files, you can define a working directory that will be uploaded together with the entry point.

If the function has custom `pip` dependencies (like in this example), add them to a `dependencies` array when constructing the `QiskitFunction` instance. The `hamiltonian_simulation/requirements-dev.txt` establishes what packages must be defined in this field:

In [2]:
dependencies = [
    "qiskit-addon-utils~=0.1.0",
    "qiskit-addon-aqc-tensor[quimb-jax]~=0.1.2",
    "mergedeep==1.3.4",
    ]

In [14]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="hamiltonian_simulation_template",
    entrypoint="hamiltonian_simulation.py",
    working_dir="./template_implementations/physics/hamiltonian_simulation/", # all files in this directory will be uploaded
    dependencies= dependencies,
)
print(template)

QiskitFunction(hamiltonian_simulation_template)


Once the instance is ready, upload it to serverless:

In [15]:
serverless.upload(template)

QiskitFunction(hamiltonian_simulation_template)

To check if the program successfully uploaded, use `serverless.list()`:

In [16]:
serverless.list()

[QiskitFunction(template_hamiltonian_simulation),
 QiskitFunction(hamiltonian_simulation_template)]

## 3. Loand and Run the Function remotely


The function template has been uploaded, so you can run it remotely with Qiskit Serverless. First, load the template by name:

In [17]:
template = serverless.load("hamiltonian_simulation_template")
print(template)

QiskitFunction(hamiltonian_simulation_template)


Next, run the template with the domain-level inputs for Hamiltonian simulation. This example specifies a 50-qubit XXZ model with random couplings, and an initial state and observable.

In [18]:
from itertools import chain
import numpy as np
from qiskit.quantum_info import SparsePauliOp

L = 50

# Generate the edge list for this spin-chain
edges = [(i, i + 1) for i in range(L - 1)]
# Generate an edge-coloring so we can make hw-efficient circuits
edges = edges[::2] + edges[1::2]

# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
Js = np.random.rand(L - 1) + 0.5 * np.ones(L - 1)

hamiltonian = SparsePauliOp.from_sparse_list(
    chain.from_iterable(
        [
            [
                ("XX", (i, j), Js[i] / 2),
                ("YY", (i, j), Js[i] / 2),
                ("ZZ", (i, j), Js[i]),
            ]
            for i, j in edges
        ]
    ),
    num_qubits=L,
)
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)

In [19]:
from qiskit import QuantumCircuit

initial_state = QuantumCircuit(L)
for i in range(L):
    if i % 2:
        initial_state.x(i)

In [20]:
job = template.run(
    dry_run=True,
    initial_state=initial_state,
    hamiltonian=hamiltonian,
    observable=observable,
    backend_name="ibm_fez",
    estimator_options={},
    aqc_evolution_time=0.2,
    aqc_ansatz_num_trotter_steps=1,
    aqc_target_num_trotter_steps=32,
    remainder_evolution_time=0.2,
    remainder_num_trotter_steps=4,
    aqc_max_iterations=300,
)
print(job.job_id)

c3744d2a-160a-4231-8bd7-8be4eef7892d


Check the status of the job:

In [21]:
job.status()

'QUEUED'

After the job is running, you can fetch logs created from the `logger.info` outputs. These can provide actionable information about the progress of the Hamiltonian simulation workflow. For example, the value of the objective function during the iterative component of AQC, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [22]:
print(job.logs())

No logs yet.


Block the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the domain-level output of Hamiltonian simulation (expectation value) and useful metadata.

In [23]:
result = job.result()

del result[
    "aqc_final_parameters"
]  # the list is too long to conveniently display here
result

{'target_bond_dimension': 5,
 'num_aqc_parameters': 816,
 'aqc_starting_fidelity': 0.9914382555614002,
 'num_iterations': 58,
 'aqc_fidelity': 0.9997886841427734,
 'twoqubit_depth': 33}

After the job completes, the entire logging output will be available.

In [24]:
print(job.logs())

2025-04-28 11:54:38,776	INFO job_manager.py:531 -- Runtime env is setting up.
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:57: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:76: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib64/python3.11/site-packages/qiskit/qpy/interface.py:346: UserWarning: The qiskit version used to generate the provided QPY file, 2.0.0, is newer than the current qiskit version 1.4.2. This may result in an error if the QPY file uses instructions not present in this current qiskit version
  warnings.warn(
hamiltonian_simulation.run_function:INFO:2025-04-28 11:55:43,512: Starting runtime service
hamiltonian_simulation.run_function:INFO:2025-04-28 11:55